# Gemini API Intro

In [ ]:
from google import genai

# looks automatically after the key
# one of GOOGLE_API_KEY and GOOGLE_API_KEY
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns from data to make smart decisions.


In [3]:
def ask_gemini(prompt, model = "gemini-2.5-flash"):
    response = client.models.generate_content(
        model=model,
        contents=prompt,
)
    
    return response

response = ask_gemini("Give me 5 some data engineering jokes, structure it in short points")
print(response.text)

Here are 5 data engineering jokes, structured in short points:

*   What's a data engineer's favorite type of data? The kind that *doesn't* change its schema without notice.

*   A business user says, "Can you just pull the data?" The data engineer replies, "Sure, right after I build a new ingestion pipeline, schema migration, and a Spark cluster."

*   How do you know a data engineer is stressed? They start muttering about "one small change" breaking an entire production pipeline.

*   A data engineer’s motto: "Our pipelines are resilient, until the source system decides to change a timestamp column to a string."

*   Why did the data engineer refuse to play hide-and-seek? Because good data is hard enough to find as it is.


In [ ]:
from pydantic import BaseModel

# knows that GenerateContentResponse is a pydantic model
# -> we can work with it in a OOP manner
isinstance(response, BaseModel)

True

In [6]:
dict(response).keys()

dict_keys(['sdk_http_response', 'candidates', 'create_time', 'model_version', 'prompt_feedback', 'response_id', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])

In [7]:
response.model_version

'gemini-2.5-flash'

In [8]:
response.sdk_http_response

HttpResponse(
  headers=<dict len=11>
)

In [9]:
response.candidates

[Candidate(
   content=Content(
     parts=[
       Part(
         text="""Here are 5 data engineering jokes, structured in short points:
 
 *   What's a data engineer's favorite type of data? The kind that *doesn't* change its schema without notice.
 
 *   A business user says, "Can you just pull the data?" The data engineer replies, "Sure, right after I build a new ingestion pipeline, schema migration, and a Spark cluster."
 
 *   How do you know a data engineer is stressed? They start muttering about "one small change" breaking an entire production pipeline.
 
 *   A data engineer’s motto: "Our pipelines are resilient, until the source system decides to change a timestamp column to a string."
 
 *   Why did the data engineer refuse to play hide-and-seek? Because good data is hard enough to find as it is."""
       ),
     ],
     role='model'
   ),
   finish_reason=<FinishReason.STOP: 'STOP'>,
   index=0
 )]

## Tokens

- basic unit of text for LLMs
- can be as short as one character or as long as one word

- tokens used for billing

Gemini free tier
- Requests per minute (RPM): 10
- Tokens per minute (TPM): 250 000
- Requests per day (RPD): 250

In [ ]:
# thinking is expensive
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=172,
  prompt_token_count=15,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=15
    ),
  ],
  thoughts_token_count=1390,
  total_token_count=1577
)

## Thinking

- hyperparameter to allocate more compute for complex tasks

In [ ]:
from google.genai import types

prompt = "Give me 5 some data engineering jokes, structure it in short points"

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0)
    ),
)

In [14]:
response.usage_metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=233,
  prompt_token_count=15,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=15
    ),
  ],
  total_token_count=248
)

## System instruction

- hyperparameter to guide model behavior

In [17]:
system_instruction = """
You are an expert in Python programming, you will always provide idiomatic code, i.e pythonic code.
So when you see my code or my question, be very critical, but answer in a SHORT and CONCISE way. 
Also be constructive to help me improve.
"""

prompt = """
Explain OOP and dunder methods
"""

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        system_instruction=system_instruction)
    )

print(response.text)

## OOP in a Nutshell

OOP (Object-Oriented Programming) is a programming paradigm based on "objects", which contain data (attributes) and code (methods) to manipulate that data. Key principles:

*   **Encapsulation:** Bundling data and methods that operate on that data within a class.
*   **Inheritance:** Creating new classes (subclasses) based on existing classes (superclasses), inheriting their attributes and methods.
*   **Polymorphism:**  The ability of objects of different classes to respond to the same method call in their own way.
*   **Abstraction:** Hiding complex implementation details and exposing only essential features.

## Dunder (Magic) Methods

Dunder methods (also called magic methods) are special methods in Python that start and end with double underscores (e.g., `__init__`, `__str__`). They provide a way to implement operator overloading and customize object behavior.

**Examples:**

*   `__init__`:  Constructor (called when an object is created).
*   `__str__`:  Str

In [19]:
metadata = response.usage_metadata
metadata

GenerateContentResponseUsageMetadata(
  candidates_token_count=308,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=308
    ),
  ],
  prompt_token_count=68,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=68
    ),
  ],
  total_token_count=376
)

In [ ]:
print(f"{metadata.candidates_token_count = }") # output
print(f"{metadata.prompt_token_count = }") # input + system instruction
print(f"{metadata.total_token_count = }")

metadata.candidates_token_count = 308
metadata.prompt_token_count = 68
metadata.total_token_count = 376


In [21]:
len(prompt.split()), len(system_instruction.split())

(5, 43)

## Temperature

- controls randomness of output -> 'creative'

- hyperparameter that can be adjusted to influence the diversity and creativity of the generated text.

In [22]:
story = "write a 2 sentence story about a gray rabbit"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=0
    ),
)

print(response.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the fear that still lingered in its twitching whiskers.



In [23]:
story = "write a 2 sentence story about a gray rabbit"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=0
    ),
)

print(response.text)

The gray rabbit twitched its nose, sensing the distant rumble of a lawnmower and immediately darted into the overgrown rose bushes. Safe within the thorny embrace, it nibbled on a fallen petal, the sweet scent masking the fear that still lingered in its twitching whiskers.



In [24]:
story = "write a 2 sentence story about a gray rabbit"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=2.0
    ),
)

print(response.text)

The gray rabbit twitched its nose, sensing danger lurking in the tall grass. It thumped its foot, warning the others before darting into the burrow, the last of its family safe from the approaching fox.



In [25]:
story = "write a 2 sentence story about a gray rabbit"

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=story,
    config=types.GenerateContentConfig(
        temperature=2.0
    ),
)

print(response.text)

The gray rabbit twitched its nose, sniffing the air for danger, before darting into the shadowed burrow. Inside, a warmth radiated from the shared body heat of her litter, a promise of safety in the cold, harsh woods.



# Multimodal input

input text and image

In [30]:
text_input = "Describe this image shortly"
image_input = {"mime_type": "image/png", "data": open("bella.png", "rb").read()}


response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=dict(
        parts=[dict(text = text_input), dict(inline_data = image_input)]
    )
)

print(response.text)

A cute, gray, fluffy rabbit is resting on a carpet while wearing a small, plush Swedish graduation cap (studentmössa) and a blue and yellow ribbon around its neck.
